In [4]:
import numpy as np
import random
import csv
import os
import json

# ====================================================================
# 0. FONCTION UTILITAIRE
# ====================================================================

def load_config(filename="input_scenario.json"):
    """Charge la configuration et les paramètres depuis un fichier JSON."""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Erreur: Le fichier de configuration '{filename}' est introuvable.")
        exit(1)
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier de configuration '{filename}' n'est pas un JSON valide.")
        exit(1)

# ====================================================================
# 1. CLASSE DE MODÉLISATION DES DONNÉES (INPUT DATA)
# ====================================================================

class InputData:
    """Modélise l'instance du problème à partir de la configuration."""

    def __init__(self, config):
        # 1. Salles
        self.SALLES = config['SALLES']
        self.SALLE_IDS = list(self.SALLES.keys())

        # 2. Créneaux Temporels (T): 6 jours * 2 périodes = 12 créneaux
        Jours = ['L', 'Ma', 'Me', 'J', 'V', 'S']
        Heures = ['Matin', 'Aprem']
        self.CRENEAUX = [f"{j}_{h}" for j in Jours for h in Heures]

        # 3. Slots de Ressource (R x T): 7 salles * 12 créneaux = 84 slots
        self.SLOTS = {f"{r}_{t}": {'Salle': r, 'Creneau': t, 'Capacite': self.SALLES[r][0]}
                      for r in self.SALLES for t in self.CRENEAUX}
        self.SLOT_IDS = list(self.SLOTS.keys())

        # 4. Classes et Tâches (C)
        CLASSES_DETAILS = config['CLASSES_DETAILS']
        self.TACHES = []
        for classe, details in CLASSES_DETAILS.items():
            effectif = details[0]
            matieres = details[1]
            for matiere in matieres:
                # L'ID de la tâche doit être unique
                self.TACHES.append({
                    'ID': f"{classe}_{matiere.replace(' ', '_')}",
                    'Classe': classe,
                    'Matiere': matiere,
                    'Effectif': effectif
                })

        self.TACHE_IDS = [t['ID'] for t in self.TACHES]
        self.N_TACHES = len(self.TACHES)  # n
        self.N_SLOTS = len(self.SLOTS)    # k
        self.tache_map = {t['ID']: t for t in self.TACHES}

        # Matrice de Compatibilité (H2 + H3)
        self.COMPATIBILITY_MATRIX = self._build_compatibility_matrix()

    def _build_compatibility_matrix(self):
        """Construit la matrice binaire de compatibilité Tâches x Slots (H2 et H3)."""
        matrix = np.zeros((self.N_TACHES, self.N_SLOTS))

        for i, tache in enumerate(self.TACHES):
            effectif = tache['Effectif']
            classe_name = tache['Classe']

            for j, slot_id in enumerate(self.SLOT_IDS):
                slot = self.SLOTS[slot_id]
                salle_name = slot['Salle']
                capacite = slot['Capacite']
                type_salle = self.SALLES[salle_name][1]

                # H3: Respect de capacité (E_i <= Cap_j)
                if effectif <= capacite:
                    is_td_class = classe_name.startswith('GMM') or classe_name.startswith('GEP')

                    # H2: Respect du type de salle
                    if is_td_class:
                        # GMM/GEP doivent être en TD
                        if type_salle == 'TD':
                            matrix[i, j] = 1
                    else:
                        # ENSTP (L1/L3) nécessitent grande salle
                        if type_salle in ['SC', 'AMPHI']:
                            matrix[i, j] = 1

        if not np.all(np.sum(matrix, axis=1) > 0):
             raise ValueError("Erreur: Certaines tâches n'ont aucune salle compatible.")
        return matrix

# ====================================================================
# 2. CLASSE DE L'ALGORITHME ACO
# (Le corps de cette classe reste inchangé par rapport à la version précédente
# hormis l'initialisation des paramètres)
# ====================================================================

class ACO_Timetabling:
    """Implémentation du MAX-MIN Ant System (MMAS)."""

    def __init__(self, data, config):
        self.data = data
        self.params = config['ACO_PARAMS']
        self._initialize_parameters(self.params)

        # Initialisation de la matrice de phéromones
        self.pheromones = self.data.COMPATIBILITY_MATRIX.copy()
        self.pheromones[self.pheromones == 1] = self.tau_max
        self.pheromones[self.pheromones == 0] = 0

    def _initialize_parameters(self, params):
        """Initialise les paramètres ACO et les poids de la fonction objectif."""
        self.alpha = params.get('alpha', 1)
        self.beta = params.get('beta', 5)
        self.rho = params.get('rho', 0.01)
        self.Q = params.get('Q', 100)
        self.tau_max = params.get('tau_max', 6)
        self.tau_min = params.get('tau_min', 0.01)
        self.w1 = params.get('w1', 1000)
        self.w2 = params.get('w2', 1)
        self.w3 = params.get('w3', 0.1)

    # --- ÉVALUATION DES SOLUTIONS ---

    def _get_f1_conflicts(self, solution):
        """Calcule f1(x): Nombre total de paires de cours de la même CLASSE au même CRÉNEAU (H1)."""
        f1_conflits = 0
        schedule_by_timeslot_and_classe = {c: [] for c in self.data.CRENEAUX}

        for tache_id, slot_id in solution.items():
            creneau = self.data.SLOTS[slot_id]['Creneau']
            classe = self.data.tache_map[tache_id]['Classe']
            schedule_by_timeslot_and_classe[creneau].append(classe)

        for _, classes_assigned in schedule_by_timeslot_and_classe.items():
            class_counts = {}
            for cls in classes_assigned:
                class_counts[cls] = class_counts.get(cls, 0) + 1

            for count in class_counts.values():
                if count > 1:
                    f1_conflits += (count - 1)

        return f1_conflits

    def calculate_objective_function(self, solution):
        """Calcule f(x) = w1*f1(x) + w2*f2(x) + w3*f3(x) (Équation 4.8)."""
        if not solution: return float('inf'), 0, 0, 0

        # 1. Calcul de f1 (Conflits Hard H1)
        f1 = self._get_f1_conflicts(solution)

        tasks_per_room = {room_id: 0 for room_id in self.data.SALLE_IDS}
        f3_occupation = 0

        # Collecte des données pour f2 et f3
        for tache_id, slot_id in solution.items():
            tache = self.data.tache_map[tache_id]
            slot = self.data.SLOTS[slot_id]
            tasks_per_room[slot['Salle']] += 1
            f3_occupation += max(0, slot['Capacite'] - tache['Effectif'])

        # 2. Calcul de f2 (Équilibre de charge des salles, Équation 4.6)
        f2_equilibre = 0
        n_sur_k = self.data.N_TACHES / len(self.data.SALLE_IDS)
        for count in tasks_per_room.values():
            f2_equilibre += (count - n_sur_k)**2

        # 3. f3 (Taux d'occupation des salles, Équation 4.7)
        f3 = f3_occupation

        # Fonction objectif globale (Équation 4.8)
        f = (self.w1 * f1) + (self.w2 * f2_equilibre) + (self.w3 * f3)
        return f, f1, f2_equilibre, f3

    # --- CONSTRUCTION DE SOLUTION PAR FOURMI ---

    def _calculate_heuristic(self, tache_id, slot_id, solution_partial):
        """Calcule l'information heuristique (eta_ij) pour une affectation."""
        tache = self.data.tache_map[tache_id]
        slot_info = self.data.SLOTS[slot_id]

        effectif = tache['Effectif']
        capacite = slot_info['Capacite']
        creneau = slot_info['Creneau']
        room_id = slot_info['Salle']
        current_classe = tache['Classe']

        # eta_cap: Proximité Capacité - Effectif (Équation 5.1)
        eta_cap = 1.0 / (1 + abs(capacite - effectif))

        # eta_conf: Éviter les conflits H1 (Équation 5.2 modifiée)
        n_conflicts_h1 = 0
        for assigned_tache_id, assigned_slot_id in solution_partial.items():
            assigned_slot_creneau = self.data.SLOTS[assigned_slot_id]['Creneau']
            assigned_classe = self.data.tache_map[assigned_tache_id]['Classe']

            if assigned_classe == current_classe and assigned_slot_creneau == creneau:
                n_conflicts_h1 += 1

        eta_conf = 1.0 / (1 + n_conflicts_h1**2)

        # eta_load: Équilibrer la charge sur la salle (Équation 5.3)
        tasks_in_room = sum(1 for _, s_id in solution_partial.items() if self.data.SLOTS[s_id]['Salle'] == room_id)
        eta_load = 1.0 / (1 + tasks_in_room)

        # Heuristique Combinée (Équation 5.4 simplifiée)
        eta_combined = eta_cap * eta_conf * eta_load

        return eta_combined

    def _transition_probability(self, tache_idx, allowed_slots_idx, solution_partial):
        """Calcule la probabilité de transition p_ij (Équation 3.6)."""
        tache_id = self.data.TACHE_IDS[tache_idx]
        numerators = []

        for slot_idx in allowed_slots_idx:
            slot_id = self.data.SLOT_IDS[slot_idx]
            tau_ij = self.pheromones[tache_idx, slot_idx]
            eta_ij = self._calculate_heuristic(tache_id, slot_id, solution_partial)

            numerator = (tau_ij**self.alpha) * (eta_ij**self.beta)
            numerators.append(numerator)

        denominator = sum(numerators)

        if denominator == 0:
            probs = np.ones(len(allowed_slots_idx)) / len(allowed_slots_idx)
        else:
            probs = np.array(numerators) / denominator

        return probs

    def construct_solution(self):
        """Une fourmi construit une solution complète (Algorithme 5.3)."""
        solution_ant = {}
        taches_restantes_idx = list(range(self.data.N_TACHES))
        random.shuffle(taches_restantes_idx)

        for tache_idx in taches_restantes_idx:
            allowed_slots_idx = np.where(self.data.COMPATIBILITY_MATRIX[tache_idx, :] == 1)[0]

            if len(allowed_slots_idx) == 0:
                return None

            probs = self._transition_probability(tache_idx, allowed_slots_idx, solution_ant)

            # Sélection du slot par roulette
            selected_slot_idx = random.choices(allowed_slots_idx, weights=probs, k=1)[0]

            tache_id = self.data.TACHE_IDS[tache_idx]
            slot_id = self.data.SLOT_IDS[selected_slot_idx]
            solution_ant[tache_id] = slot_id

        return solution_ant

    # --- MISE À JOUR DES PHÉROMONES (MMAS) ---

    def update_pheromones(self, best_solution_cycle, f_best_cycle, f_best_global):
        """Mise à jour des phéromones (Évaporation + Dépôt + Bornes MMAS)."""

        # 1. Évaporation (Équation 3.7)
        self.pheromones = (1 - self.rho) * self.pheromones

        # 2. Dépôt de phéromone par la meilleure solution du cycle
        delta_tau = self.Q / (f_best_cycle + 1.0)

        for tache_id, slot_id in best_solution_cycle.items():
            tache_idx = self.data.TACHE_IDS.index(tache_id)
            slot_idx = self.data.SLOT_IDS.index(slot_id)
            self.pheromones[tache_idx, slot_idx] += delta_tau

        # 3. Application des bornes MAX-MIN (Équation 3.11)
        compatible_indices = np.where(self.data.COMPATIBILITY_MATRIX == 1)

        for i, j in zip(*compatible_indices):
            self.pheromones[i, j] = max(self.tau_min, min(self.tau_max, self.pheromones[i, j]))

        # S'assurer que les chemins non compatibles restent à 0
        non_compatible_indices = np.where(self.data.COMPATIBILITY_MATRIX == 0)
        self.pheromones[non_compatible_indices] = 0

    # --- BOUCLE PRINCIPALE ---

    def solve(self):
        """Boucle principale de l'algorithme ACO."""
        n_ants = self.params.get('m', 30)
        max_iterations = self.params.get('MaxIterations', 100)

        best_solution_global = {}
        f_best_global = float('inf')

        print(f"Démarrage ACO | Tâches (n): {self.data.N_TACHES}, Slots (k): {self.data.N_SLOTS}, Fourmis (m): {n_ants}, Iterations: {max_iterations}")
        print("-" * 50)

        for iteration in range(max_iterations):
            f_best_cycle = float('inf')
            best_solution_cycle = {}

            for _ in range(n_ants):
                solution_ant = self.construct_solution()

                if solution_ant is None:
                    f_ant = float('inf')
                else:
                    f_ant, _, _, _ = self.calculate_objective_function(solution_ant)

                # Mise à jour du meilleur du cycle
                if f_ant < f_best_cycle:
                    f_best_cycle = f_ant
                    best_solution_cycle = solution_ant

                # Mise à jour du meilleur global
                if f_ant < f_best_global:
                    f_best_global = f_ant
                    best_solution_global = solution_ant

            if best_solution_cycle:
                self.update_pheromones(best_solution_cycle, f_best_cycle, f_best_global)

            f_global_components = self.calculate_objective_function(best_solution_global)
            print(f"Iter {iteration+1:03d}: f_cycle={f_best_cycle:.2f} | f_global={f_best_global:.2f} (f1={f_global_components[1]}, f2={f_global_components[2]:.2f}, f3={f_global_components[3]:.2f})")

            if f_best_global == 0.0:
                break

        return best_solution_global, f_best_global

# ====================================================================
# 3. CLASSE D'EXPORT DES RÉSULTATS (CSV)
# (Inchangée, gère la sortie des résultats)
# ====================================================================

class DataExporter:
    """Gère l'export des résultats de l'ACO au format CSV."""

    def __init__(self, data):
        self.data = data
        self.output_dir = "aco_results"
        os.makedirs(self.output_dir, exist_ok=True)

    def export_pheromones(self, pheromone_matrix):
        """Exporte la matrice de phéromones finale (Tâches x Slots)."""
        filename = os.path.join(self.output_dir, "pheromone_matrix.csv")
        header = ["Tache"] + self.data.SLOT_IDS

        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(header)

            for i, tache_id in enumerate(self.data.TACHE_IDS):
                row = [tache_id] + [f"{val:.6f}" for val in pheromone_matrix[i, :].tolist()]
                writer.writerow(row)
        print(f"Exporté la matrice de phéromones vers {filename}")

    def export_timetable(self, solution):
        """Exporte l'emploi du temps final (Timetable) au format lisible."""
        filename = os.path.join(self.output_dir, "emploi_du_temps.csv")

        header = ["Jour", "Période", "Salle", "Capacite_Salle", "Classe", "Matiere", "Effectif_Classe", "Slot_ID", "Tache_ID"]
        timetable_data = []

        for tache_id, slot_id in solution.items():
            tache = self.data.tache_map[tache_id]
            slot = self.data.SLOTS[slot_id]

            jour, periode = slot['Creneau'].split('_')

            timetable_data.append([
                jour, periode, slot['Salle'], slot['Capacite'],
                tache['Classe'], tache['Matiere'], tache['Effectif'],
                slot_id, tache_id
            ])

        timetable_data.sort(key=lambda x: (x[0], x[1], x[2]))

        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(timetable_data)
        print(f"Exporté l'emploi du temps vers {filename}")

    def export_load_distribution(self, solution):
        """Exporte la répartition de charge (f2) et le taux d'occupation (f3)."""
        filename = os.path.join(self.output_dir, "repartition_charge.csv")

        tasks_per_room = {r: [] for r in self.data.SALLE_IDS}
        for tache_id, slot_id in solution.items():
            tasks_per_room[self.data.SLOTS[slot_id]['Salle']].append(tache_id)

        header = ["Salle", "Capacite", "Type_Salle", "Nb_Cours_Affectes", "Taches_IDs", "Gaspillage_Moyen"]
        load_data = []

        for salle_id in self.data.SALLE_IDS:
            nb_cours = len(tasks_per_room[salle_id])
            capacite = self.data.SALLES[salle_id][0]
            type_salle = self.data.SALLES[salle_id][1]

            gaspillage_total = 0
            for tache_id in tasks_per_room[salle_id]:
                effectif = self.data.tache_map[tache_id]['Effectif']
                gaspillage_total += max(0, capacite - effectif)

            gaspillage_moyen = (gaspillage_total / nb_cours) if nb_cours > 0 else 0

            load_data.append([
                salle_id, capacite, type_salle, nb_cours,
                "; ".join(tasks_per_room[salle_id]),
                f"{gaspillage_moyen:.2f}"
            ])

        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(load_data)
        print(f"Exporté la répartition de charge vers {filename}")


# ====================================================================
# 4. EXÉCUTION PRINCIPALE
# ====================================================================

if __name__ == '__main__':
    # 1. Chargement de la configuration
    CONFIG_FILE = "input_scenario.json"
    config = load_config(CONFIG_FILE)

    print(f"--- Préparation du Scénario chargé depuis {CONFIG_FILE} ---")
    data = InputData(config)

    # 2. Initialisation et exécution de l'ACO
    print("\n--- Démarrage de l'Optimisation par Colonies de Fourmis (ACO) ---")
    aco = ACO_Timetabling(data, config)
    final_solution, final_cost = aco.solve()

    # 3. Finalisation et Export des Résultats
    print("\n--- Finalisation et Export des Résultats ---")
    exporter = DataExporter(data)

    if final_solution and final_cost != float('inf'):
        f_final, f1_final, f2_final, f3_final = aco.calculate_objective_function(final_solution)

        print("\n=============================================")
        print("RÉSUMÉ DES RÉSULTATS")
        print("=============================================")
        print(f"Coût Total Final (f(x)): {f_final:.2f}")
        print(f"Violations Hard (f1 - Conflits H1) : {f1_final}")
        print(f"Équilibre de charge (f2) : {f2_final:.2f}")
        print(f"Gaspillage de capacité (f3) : {f3_final:.2f}")

        exporter.export_pheromones(aco.pheromones)
        exporter.export_timetable(final_solution)
        exporter.export_load_distribution(final_solution)
    else:
        print("Échec: Aucune solution valide n'a pu être construite par l'ACO.")

--- Préparation du Scénario chargé depuis input_scenario.json ---

--- Démarrage de l'Optimisation par Colonies de Fourmis (ACO) ---
Démarrage ACO | Tâches (n): 40, Slots (k): 84, Fourmis (m): 30, Iterations: 150
--------------------------------------------------
Iter 001: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 002: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 003: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 004: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 005: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 006: f_cycle=83.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 007: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 008: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 009: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 010: f_cycle=81.43 | f_global=81.43 (f1=0, f2=47.43, f3=340.00)
Iter 011: f_cycle=81.43 | f_global=81.43 (f1=0